# Test our hypotheses: AUC measure
To be really prudent, we replicate the previous analysis of our hypotheses but for the AUC metrics for each of the models.

Burnham & Anderson (2002) present heuristics to interpret the $\Delta$ AIC.
- $\Delta AIC = 0-2$: litle to distiniguish the models
- $\Delta AIC = 4-7$: considerably less support for the model with higher AIC
- $\Delta AIC > 10$: essentially no support for the model with higher AIC

Burnham, K. P., & Anderson, D. R. (2004). Multimodel Inference: Understanding AIC and BIC in Model Selection, 33(2), 261–304. http://doi.org/10.1177/0049124104268644

Heuristics to interpret the $\Delta$ BIC.
- $\Delta BIC = 0-2$ : litle to distiniguish the models
- $\Delta BIC = 2-6$ : positive evidence against the model with higher BIC
- $\Delta BIC = 6-10$ : strong evidence against the model with higher BIC
- $\Delta BIC > 10$ : very strong evidence against the model with higher BIC

Kass, Robert E.; Raftery, Adrian E. (1995), "Bayes Factors", Journal of the American Statistical Association, 90 (430): 773–795, doi:10.2307/2291091

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import cauchy
from scipy.optimize import minimize

Define our models. We must do it here, where `data` is in the local scope, in order to call the `nll` method as a static method in order to use `scipy.optimize.minimize`.

Bear in mind that we have changed the `loc` parameters below because these predictions are for AUC, not $\log(k)$. More specifically, we just added a minus sign.

In [2]:
class Model:

    def calc_aic(self):
        return -2*self.ll + 2*self.free_params

    def calc_bic(self):
        return -2*self.ll + np.log(self.n_observations)*self.free_params

    def fit(self, data):
        '''Find parameters which minimise the negative log likelihood.'''
        self.data = data
        self.n_observations = len(self.data['delta_food'])
        if not self.bounds:
            result = minimize(self.nll, self.x0,
                              method='Nelder-Mead',
                              options={'disp': True},
                              args=data)
        else:
            result = minimize(self.nll, self.x0,
                              method='L-BFGS-B', bounds=self.bounds,
                              options={'disp': True},
                              args=data)
        self.mlparams = result.x
        self.nll = result.fun
        self.ll = -self.nll
        self.AIC = self.calc_aic()
        self.BIC = self.calc_bic()
        return self


class H1(Model):
    """Our control (trait-only) model which assumes zero change in AUC"""

    name = "1. Trait only"
    x0 = [0.05]
    free_params = len(x0)
    bounds = None

    @staticmethod
    def nll(params, data):
        return (-sum(cauchy.logpdf(data['delta_food'], loc=0, scale=params[0]) +
                     cauchy.logpdf(data['delta_money'], loc=0, scale=params[0]) +
                     cauchy.logpdf(data['delta_music'], loc=0, scale=params[0])))


class H2(Model):
    """In-domain model"""

    name = "2. In-domain"
    x0 = [+0.25, 0.05]
    free_params = len(x0)
    bounds = None

    @staticmethod
    def nll(params, data):
        return -sum(cauchy.logpdf(data['delta_food'], loc=-params[0], scale=params[1]) +
                   cauchy.logpdf(data['delta_money'], loc=0, scale=params[1]) +
                   cauchy.logpdf(data['delta_music'], loc=0, scale=params[1]))


class H3(Model):
    """Monetary fungibility model"""

    name = "3. Monetary fungibility"
    x0 = [0.25, 0.05]
    free_params = len(x0)
    bounds = None

    @staticmethod
    def nll(params, data):
        return -sum(cauchy.logpdf(data['delta_food'], loc=-params[0], scale=params[1]) +
               cauchy.logpdf(data['delta_money'], loc=-params[0], scale=params[1]) +
               cauchy.logpdf(data['delta_music'], loc=0, scale=params[1]))


class H4(Model):
    """Negative spillover model"""

    name = "4. Negative spillover"
    x0 = [0.25, -0.1, 0.05]
    free_params = len(x0)
    bounds = [(0., None), (None, 0.0), (0.0001, None)]

    @staticmethod
    def nll(params, data):
        return -sum(cauchy.logpdf(data['delta_food'], loc=-params[0], scale=params[2]) +
               cauchy.logpdf(data['delta_money'], loc=-params[1], scale=params[2]) +
               cauchy.logpdf(data['delta_music'], loc=-params[1], scale=params[2]))


class H5(Model):
    """Spillover model"""

    name = "5. Spillover"
    x0 = [+0.25, +0.1, 0.05]
    free_params = len(x0)
    bounds = None

    @staticmethod
    def nll(params, data):
        return -sum(cauchy.logpdf(data['delta_food'], loc=-params[0], scale=params[2]) +
               cauchy.logpdf(data['delta_money'], loc=-params[1], scale=params[2]) +
               cauchy.logpdf(data['delta_music'], loc=-params[1], scale=params[2]))


class H6(Model):
    """State-only model"""

    name = "6. State-only"
    x0 = [+0.25, 0.05]
    free_params = len(x0)
    bounds = None

    @staticmethod
    def nll(params, data):
        return -sum(cauchy.logpdf(data['delta_food'], loc=-params[0], scale=params[1]) +
               cauchy.logpdf(data['delta_money'], loc=-params[0], scale=params[1]) +
               cauchy.logpdf(data['delta_music'], loc=-params[0], scale=params[1]))

In [3]:
def long_to_wide(df, target_param):
    '''Convert long-form data to wide-form data'''
    # pivot long to wide
    df = df.set_index('id')
    df = df.pivot_table(index='id', columns=['commodity', 'condition'], values=target_param)
    # collapse column multi-index 
    df.columns = [' '.join(col).strip() for col in df.columns.values]
    # set index to a column
    df = df.reset_index()
    return df

In [4]:
def evaluate_hypotheses(filename, discount_func_name):
    data_long = pd.read_csv(filename)
    data_wide = long_to_wide(data_long, target_param='AUC')

    delta_logk_food = (data_wide['food F'] - data_wide['food C']).values
    delta_logk_money = (data_wide['money F'] - data_wide['money C']).values
    delta_logk_music = (data_wide['music F'] - data_wide['music C']).values

    data = {'delta_food': delta_logk_food,
            'delta_money': delta_logk_money, 
            'delta_music': delta_logk_music}

    models = [H1(), H2(), H3(), H4(), H5(), H6()]

    models = [model.fit(data) for model in models]

    for model in models:
        print(f'model: {model.name}, params: {model.mlparams}')

    # Summarise in a dataframe ====================================
    def W(x):
        return np.exp(-x/2)

    # summarise data in a DataFrame
    aic = np.array([model.AIC for model in models])
    delta_aic = aic - min(aic)
    waic = W(aic)/sum(W(aic))

    bic = np.array([model.BIC for model in models])
    delta_bic = bic - min(bic)
    wbic = W(bic)/sum(W(bic))

    info = {'model': [model.name for model in models],
            'n': [model.free_params for model in models],
            'LL': [model.ll for model in models],
            'AIC': [model.AIC for model in models],
            'deltaAIC': delta_aic,
            'AIC weight': waic,
            'BIC': [model.BIC for model in models],
            'deltaBIC': delta_bic,
            'BIC weight': wbic,
            'parameters': [model.mlparams for model in models]}

    results = pd.DataFrame.from_dict(info)
    results.to_csv(f'model_comparison_AUC_{discount_func_name}.csv', index=False)
    return results

## Evaluate for the exponential discount function

In [5]:
Exponential_results = evaluate_hypotheses('parameter_estimation_Exponential.csv', 'Exponential')
Exponential_results

Optimization terminated successfully.
         Current function value: 61.110248
         Iterations: 14
         Function evaluations: 28
Optimization terminated successfully.
         Current function value: 50.097993
         Iterations: 38
         Function evaluations: 74
Optimization terminated successfully.
         Current function value: 50.405436
         Iterations: 35
         Function evaluations: 69
Optimization terminated successfully.
         Current function value: 43.882024
         Iterations: 63
         Function evaluations: 119
Optimization terminated successfully.
         Current function value: 47.576507
         Iterations: 35
         Function evaluations: 68
model: 1. Trait only, params: [0.13554688]
model: 2. In-domain, params: [0.19348746 0.14118666]
model: 3. Monetary fungibility, params: [0.08970892 0.13595648]
model: 4. Negative spillover, params: [0.19348104 0.         0.14117536]
model: 5. Spillover, params: [0.18980369 0.0578481  0.13893288]
model: 

,model,n,LL,AIC,deltaAIC,AIC weight,BIC,deltaBIC,BIC weight,parameters
0,1. Trait only,1,-61.110248,124.220496,30.456448,2.256558e-07,126.132519,26.632402,0.000001,[0.1355468750000003]
1,2. In-domain,2,-50.097993,104.195985,10.431937,5.031688e-03,108.020031,8.519914,0.011747,"[0.19348746003149048, 0.14118666419231807]"
2,3. Monetary fungibility,2,-50.405436,104.810872,11.046824,3.699923e-03,108.634918,9.134801,0.008638,"[0.08970891982935343, 0.13595647730207946]"
3,4. Negative spillover,3,-50.097992,106.195985,12.431937,1.851055e-03,111.932054,12.431937,0.001661,"[0.19348103875250722, 0.0, 0.14117535618152705]"
4,5. Spillover,3,-43.882024,93.764048,0.000000,9.267873e-01,99.500117,0.000000,0.831742,"[0.18980369495823488, 0.057848101363559, 0.138..."
5,6. State-only,2,-47.576507,99.153014,5.388966,6.262980e-02,102.977060,3.476943,0.146211,"[0.07722085174173014, 0.13528028701432054]"


According to the AIC:
- the best model is the `spillover` hypothesis. 
- there is consideratbly less support for the `state-only` hypothesis.
- there is essentially no support for the remaining hypotheses.

The story is much the same according to the BIC metric:
- the best model is the `spillover` hypothesis. 
- there is evidence against the `state-only` hypothesis.
- there is strong evidence against the `in-domain` and `monetary fungibility` hypotheses.
- there is very strong evidence against the `negative spillover` and `trait-only` hypotheses.

## Evaluate for the hyperbolic discount function

In [6]:
Hyperbolic_results = evaluate_hypotheses('parameter_estimation_Hyperbolic.csv', 'Hyperbolic')
Hyperbolic_results

Optimization terminated successfully.
         Current function value: 57.273710
         Iterations: 14
         Function evaluations: 28
Optimization terminated successfully.
         Current function value: 36.190305
         Iterations: 32
         Function evaluations: 60
Optimization terminated successfully.
         Current function value: 42.685891
         Iterations: 41
         Function evaluations: 78
Optimization terminated successfully.
         Current function value: 29.855748
         Iterations: 71
         Function evaluations: 130
Optimization terminated successfully.
         Current function value: 40.727859
         Iterations: 40
         Function evaluations: 76
model: 1. Trait only, params: [0.14054688]
model: 2. In-domain, params: [0.27387094 0.13045651]
model: 3. Monetary fungibility, params: [0.11496806 0.13791024]
model: 4. Negative spillover, params: [0.27390925 0.         0.13045772]
model: 5. Spillover, params: [0.27339816 0.05498741 0.12876563]
model: 

,model,n,LL,AIC,deltaAIC,AIC weight,BIC,deltaBIC,BIC weight,parameters
0,1. Trait only,1,-57.273710,116.547421,50.835925,9.083195e-12,118.459444,47.011879,6.099030e-11,[0.14054687500000035]
1,2. In-domain,2,-36.190305,76.380611,10.669115,4.790166e-03,80.204657,8.757092,1.236466e-02,"[0.27387093980505517, 0.13045651411928677]"
2,3. Monetary fungibility,2,-42.685891,89.371782,23.660285,7.233587e-06,93.195828,21.748262,1.867176e-05,"[0.1149680559618323, 0.13791024222335457]"
3,4. Negative spillover,3,-36.190305,78.380610,12.669114,1.762204e-03,84.116679,12.669114,1.748628e-03,"[0.27390924750224305, 0.0, 0.13045771906221798]"
4,5. Spillover,3,-29.855748,65.711496,0.000000,9.933891e-01,71.447565,0.000000,9.857357e-01,"[0.27339816352430174, 0.054987405052453725, 0...."
5,6. State-only,2,-40.727859,85.455718,19.744221,5.125264e-05,89.279764,17.832198,1.322963e-04,"[0.09049939507385628, 0.13784416473470645]"


According to the AIC:
- the best model is the `spillover` hypothesis. 
- there is essentially no support for the remaining hypotheses.

The story is much the same according to the BIC metric:
- the best model is the `spillover` hypothesis. 
- there is strong evidence against the `in-domain` hypothesis.
- there is very strong evidence against the remaining hypotheses.

## Evaluate for the Modified Rachlin discount function

In [7]:
ModifiedRachlin_results = evaluate_hypotheses('parameter_estimation_ModifiedRachlin.csv', 'ModifiedRachlin')
ModifiedRachlin_results

Optimization terminated successfully.
         Current function value: 36.789011
         Iterations: 14
         Function evaluations: 28
Optimization terminated successfully.
         Current function value: 19.578292
         Iterations: 41
         Function evaluations: 77
Optimization terminated successfully.
         Current function value: 26.543160
         Iterations: 36
         Function evaluations: 70
Optimization terminated successfully.
         Current function value: 15.743490
         Iterations: 61
         Function evaluations: 113
Optimization terminated successfully.
         Current function value: 25.235905
         Iterations: 34
         Function evaluations: 65
model: 1. Trait only, params: [0.125]
model: 2. In-domain, params: [0.22176991 0.12110567]
model: 3. Monetary fungibility, params: [0.09060064 0.1293568 ]
model: 4. Negative spillover, params: [0.22177022 0.         0.12108618]
model: 5. Spillover, params: [0.22097639 0.04107258 0.11852276]
model: 6. St

,model,n,LL,AIC,deltaAIC,AIC weight,BIC,deltaBIC,BIC weight,parameters
0,1. Trait only,1,-36.789011,75.578022,38.091041,4.954183e-09,77.490045,34.266995,3.082884e-08,[0.12500000000000028]
1,2. In-domain,2,-19.578292,43.156585,5.669604,5.434977e-02,46.980631,3.757581,1.300147e-01,"[0.22176991166741072, 0.1211056708846728]"
2,3. Monetary fungibility,2,-26.543160,57.086320,19.599339,5.133271e-05,60.910366,17.687316,1.227973e-04,"[0.09060064188815793, 0.1293567984765832]"
3,4. Negative spillover,3,-19.578291,45.156583,7.669602,1.999418e-02,50.892652,7.669602,1.838687e-02,"[0.22177022326489343, 0.0, 0.12108617685070641]"
4,5. Spillover,3,-15.743490,37.486981,0.000000,9.254150e-01,43.223050,0.000000,8.510218e-01,"[0.2209763909413237, 0.04107257923979059, 0.11..."
5,6. State-only,2,-25.235905,54.471811,16.984830,1.897263e-04,58.295857,15.072807,4.538604e-04,"[0.06791818185993087, 0.1236953317702319]"


According to the AIC:
- the best model is the `spillover` hypothesis. 
- there is considerably less support for the `in-domain` hypothesis.
- there is essentially no support for the remaining hypotheses.

The story is much the same according to the BIC metric:
- the best model is the `spillover` hypothesis. 
- there is positive evidence against the `in-domain` hypothesis.
- there is strong evidence against the `negative spillover` hypothesis.
- there is very strong evidence against the remaining hypotheses.

## Evaluate for the Hyperboloid discount function

In [8]:
ModifiedRachlin_results = evaluate_hypotheses('parameter_estimation_Hyperboloid.csv', 'Hyperboloid')
ModifiedRachlin_results

Optimization terminated successfully.
         Current function value: 58.209051
         Iterations: 14
         Function evaluations: 28
Optimization terminated successfully.
         Current function value: 37.160640
         Iterations: 31
         Function evaluations: 60
Optimization terminated successfully.
         Current function value: 45.548172
         Iterations: 37
         Function evaluations: 70
Optimization terminated successfully.
         Current function value: 32.236037
         Iterations: 74
         Function evaluations: 137
Optimization terminated successfully.
         Current function value: 43.528181
         Iterations: 36
         Function evaluations: 68
model: 1. Trait only, params: [0.14375]
model: 2. In-domain, params: [0.29688905 0.1359658 ]
model: 3. Monetary fungibility, params: [0.11499083 0.14494342]
model: 4. Negative spillover, params: [0.29689517 0.         0.13595708]
model: 5. Spillover, params: [0.29682599 0.0528158  0.1357205 ]
model: 6. 

,model,n,LL,AIC,deltaAIC,AIC weight,BIC,deltaBIC,BIC weight,parameters
0,1. Trait only,1,-58.209051,118.418102,47.946027,3.776237e-11,120.330125,44.121981,2.478813e-10,[0.14375000000000035]
1,2. In-domain,2,-37.160640,78.321279,7.849205,1.922975e-02,82.145325,5.937182,4.852523e-02,"[0.29688905141956545, 0.13596580489829616]"
2,3. Monetary fungibility,2,-45.548172,95.096345,24.624270,4.378389e-06,98.920391,22.712247,1.104863e-05,"[0.11499082580900147, 0.14494341738883781]"
3,4. Negative spillover,3,-37.160639,80.321279,9.849204,7.074230e-03,86.057348,9.849204,6.862505e-03,"[0.29689516745619077, 0.0, 0.13595707601632898]"
4,5. Spillover,3,-32.236037,70.472075,0.000000,9.736586e-01,76.208144,0.000000,9.445179e-01,"[0.29682598819143075, 0.05281579528817026, 0.1..."
5,6. State-only,2,-43.528181,91.056362,20.584287,3.300544e-05,94.880408,18.672264,8.328744e-05,"[0.09058799476290369, 0.14372554030978243]"


According to the AIC:
- the best model is the `spillover` hypothesis. 
- there is essentially no support for the remaining hypotheses.

The story is much the same according to the BIC metric:
- the best model is the `spillover` hypothesis. 
- there is positive evidence against the `in-domain` hypothesis.
- there is strong evidence against the `negative spillover` hypothesis.
- there is very strong evidence against the remaining hypotheses.

## Summary
There is a clear pattern of results. For all discount functions examined, for both AIC and BIC metrics, the Spillover Hypothesis was the best account of the data. The next best hypothesis was the In-Domain Hypothesis, however there was considerably less support for this as compared to the Spillover Hypothesis. The data showed strong or very strong evidence against the remaining hypotheses.